In [1]:
import os
from dotenv import load_dotenv
print(load_dotenv('./.env'))
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

True


In [2]:
gemini = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-preview-05-20", 
    temperature=0.4,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=os.getenv('GEMINI_API_KEY_JY'),
)

gpt = ChatOpenAI(
    model="gpt-4o",
    temperature=0.4,
    api_key=os.getenv('OPENAI_API_KEY'),
)


In [3]:
df = pd.read_csv('data/samples.csv')
print(df.head())

                                                                        user_prompt  \
0                      우리 공군 특수비행팀 블랙이글스는 이번에 축하비행을 한 영국 공군 레드 애로우즈와 깊은 인연을 가지고 있다.   
1                                    나머지 6억5000만원은 수취인이 불명확해 경찰의 계좌추적 결과를 참고하기로 했다.   
2                                          모든 백내장 수술을 입원 치료로 인정해서는 안 된다는 대법원의 판결이다.   
3  한편 조합은 지난 2017년 시공사로 선정된 대우건설과 설계변경에 따른 공사비 증액 문제로 갈등을 빚다가 지난해 12월 시공사 계약을 해지했다.   
4                                           박물관에서는 80대 할머니 한 명이 지팡이를 짚은 채 둘러보고 있었다.   

         output  
0  사실형,긍정,현재,확실  
1  사실형,긍정,과거,확실  
2  사실형,긍정,현재,확실  
3  사실형,긍정,과거,확실  
4  사실형,긍정,과거,확실  


In [ ]:
prompt = PromptTemplate.from_template(
"""
당신은 주어진 '특별 규칙'에 따라 한국어 문장을 분류하는 정밀 라벨링 전문가입니다.
일반적인 언어 규칙보다 아래에 명시된 **특별 규칙과 판례를 최우선으로 적용**하여 분류를 수행해야 합니다.

# 기본 지침
1. 모든 문장은 '유형, 극성, 시제, 확실성' 네 가지 속성으로 분류합니다.
2. 출력 형식은 반드시 '번호. 유형,극성,시제,확실성' 순서를 지키고, 다른 설명은 절대 추가하지 않습니다.

# 분류 정의
1) 유형
- 사실형: 객관적 사건, 정보, 통계, 현상, 또는 확정된 계획/예정을 전달하는 문장. (가장 기본)
- 추론형: 원인 분석, 해석, 주관적 평가/감정이 포함된 문장.
- 대화형: 직접 인용문("..."), 또는 뉴스 브리핑/발표 형식의 경어체(~습니다) 문장.
- 예측형: 불확실한 미래 전망이나 단순 추측을 나타내는 문장.

2) 극성
- 긍정: 긍정적 내용, 또는 내용의 좋고 나쁨과 관계없이 객관적 정보를 중립적으로 전달하는 모든 문장.
- 부정: 범죄, 재난, 전쟁, 명시적인 비판 등 극히 제한적인 부정적 사건.

3) 시제
- 과거/현재/미래: 문장의 서술어(문법)가 아닌, **문장이 설명하는 사건이 발생하는 실제 시간**을 기준으로 판단.

4) 확실성
- 확실: 확정된 사실, 계획, 통계.
- 불확실: 추측, 의견, 전망. (~로 보인다, ~일 수도)

# ★★★ 특별 규칙 및 판례 (가장 중요) ★★★

이 규칙을 모든 판단의 최우선 기준으로 삼으시오.

**1. 극성 판례: '객관적 정보 전달'은 내용과 무관하게 '긍정'이다.**
 - 예시 문장: "지난해 관광객이 25%나 줄어 558만여 명에 그쳤다."
 - 판결: 내용이 부정적(감소)이지만, 객관적 수치를 전달하므로 **긍정**으로 분류한다.
 - 예시 문장: "신규 가입자는 앱 이용에 불편을 겪어야 했다."
 - 판결: '불편'은 부정적 경험이지만, '겪었다'는 현상을 전달하므로 **긍정**으로 분류한다.

**2. 시제 판례: '사건 발생 시점'이 문법보다 우선한다.**
 - 예시 문장: "신청 기간은 이달 14일까지다."
 - 판결: 문법은 현재형(-이다)이지만, '14일'이라는 사건 시점은 미래이므로 **미래**로 분류한다.

**3. 유형 판례 1: '확정된 계획/예정'은 '사실형'이다.**
 - 예시 문장: "'기생충' 드라마에는 봉 감독이 참여할 예정이다."
 - 판결: '~ㄹ 예정이다' 표현이 있지만, 확정된 계획을 정보로 전달하므로 '예측형'이 아닌 **사실형**으로 분류한다.
 - 예시 문장: "새해 들어 이런 판도에 변화가 나타날 조짐이다."
 - 판결: '조짐'이라는 표현은 미래를 암시하지만, 현재 관찰되는 '조짐'이라는 사실을 전달하므로 **사실형**으로 분류한다.

**4. 유형 판례 2: '뉴스/브리핑 경어체'는 '대화형'이다.**
 - 예시 문장: "강한 순환매매 장세가 펼쳐지고 있습니다."
 - 판결: 따옴표가 없지만, 청중에게 말하는 듯한 경어체(~습니다)로 끝났으므로 **대화형**으로 분류한다.

**5. 유형 판례 3: '주관적 평가/해석'이 포함되면 '추론형'이다.**
 - 예시 문장: "볼커의 뚝심은 1981년부터 효과가 나타났다."
 - 판결: '뚝심'이 '효과'를 냈다는 것은 사실에 대한 해석이므로 **추론형**으로 분류한다.
 - 예시 문장: "한국 아동문학계를 놀라게 한 사건이었다."
 - 판결: '놀라게 했다'는 것은 사건에 대한 평가/영향을 해석한 것이므로 **추론형**으로 분류한다.


문제:```{questions}```
"""
)

In [56]:
import random

n_rows = len(df)
rand_n = random.randint(5, 100)

sample_df = df.sample(rand_n)

print(len(sample_df))

15


In [70]:
questions = ""
for idx, i in enumerate(sample_df['user_prompt']):
    questions += str(idx+1) + ". " + i
    if idx != len(sample_df['user_prompt'])-1:
        questions += "\n"
print(questions)

1. 특히 탈중앙화 금융(디파이) 디앱에 주목할 필요가 있다.
2. 지난해 전체 관광객이 2% 증가하는 동안 한국에서 온 관광객은 2018년(753만여 명)보다 25%나 줄어 558만여 명에 그쳤다.
3. 예술경영지원센터가 최근 발표한 ＇2019 미술시장 실태조사＇에 따르면 국내 미술시장 규모는 5000억원대 문턱을 넘지 못하고 다시 4000억원대 중반으로 주저앉았다.
4. ＇자유한국당은 뭐하나, 이런 사람 영입 안 하고.＇
5. 세계는 물론 한국 아동문학계를 놀라게 한 사건이었다.
6. 신청 기간은 이달 14일까지다.
7. 아델 아비딘 ＇청소＇는 먼지 묻은 사람들을 일렬로 세워두고 물대포로 씻기는 영상 작품이다.
8. 신규 가입자는 물론 기존 하나은행 고객들까지 앱 이용에 불편을 겪어야 했다.
9. 주위에 버려진 쓰레기들이 아쉬워 몇 개 주워 다음 코스로 이동했다.
10. 새해 들어 이런 판도에 변화가 나타날 조짐이다.
11. ＇기생충＇ 드라마에는 봉 감독이 책임 프로듀서로 참여할 예정이다.
12. 아프리카돼지열병이나 산불과 같은 재난이 발생했을 때 수차례 현장을 직접 찾아 ＇안전 총리＇라는 별명을 얻었던 이낙연 전 총리의 행보와 비교된다는 지적이 나온다.
13. 미술 전문 도슨트 투어 그룹인 ＇소통하는 그림 연구소＇는 전시 설명을 맡을 예정이다.
14. 볼커의 뚝심은 1981년 중반부터 효과가 나타났다.
15. 비트코인이 1,100만원 돌파를 앞두고 잠시 쉬어가는 사이 알트코인 내 강한 순환매매 장세가 펼쳐지고 있습니다.


In [81]:
chain = prompt | gemini

response = chain.invoke({'questions':questions})

print(response.content)

1. 추론형,긍정,현재,불확실
2. 사실형,긍정,과거,확실
3. 사실형,긍정,과거,확실
4. 대화형,긍정,현재,확실
5. 추론형,긍정,과거,불확실
6. 사실형,긍정,미래,확실
7. 사실형,긍정,현재,확실
8. 사실형,긍정,과거,확실
9. 추론형,긍정,과거,불확실
10. 사실형,긍정,현재,불확실
11. 사실형,긍정,미래,확실
12. 추론형,긍정,현재,불확실
13. 사실형,긍정,미래,확실
14. 추론형,긍정,과거,불확실
15. 대화형,긍정,현재,확실


In [82]:
response_split = response.content.split('\n')

for idx, line in enumerate(response_split):
    if line != line.strip():
        print(f"⚠️ {idx}번째 문장에 앞/뒤 공백이 있습니다.")
        print(f"원본 : '{line}'")
        print(f"수정 : '{line.strip()}'")
        print("--------------------------------")

In [83]:
incorrect = []
for idx, i in enumerate(sample_df['output']):
    output = str(idx+1) + ". " + i
    if response_split[idx] != output:
        incorrect.append(idx)
        
print(f"맞춘문제: {len(sample_df) - len(incorrect)}")
print("틀린문제: ", len(incorrect)/len(sample_df)*100, "%")
for i in incorrect:
    print(f"{i} : {sample_df['user_prompt'].iloc[i]}")
    print(f"정답 : {i+1}. {sample_df['output'].iloc[i]}")
    print(f"오답 : {response_split[i]}")
    print("--------------------------------")

맞춘문제: 7
틀린문제:  53.333333333333336 %
0 : 특히 탈중앙화 금융(디파이) 디앱에 주목할 필요가 있다.
정답 : 1. 추론형,긍정,현재,확실
오답 : 1. 추론형,긍정,현재,불확실
--------------------------------
3 : ＇자유한국당은 뭐하나, 이런 사람 영입 안 하고.＇
정답 : 4. 대화형,긍정,현재,불확실
오답 : 4. 대화형,긍정,현재,확실
--------------------------------
4 : 세계는 물론 한국 아동문학계를 놀라게 한 사건이었다.
정답 : 5. 사실형,긍정,과거,확실
오답 : 5. 추론형,긍정,과거,불확실
--------------------------------
8 : 주위에 버려진 쓰레기들이 아쉬워 몇 개 주워 다음 코스로 이동했다.
정답 : 9. 사실형,긍정,과거,확실
오답 : 9. 추론형,긍정,과거,불확실
--------------------------------
9 : 새해 들어 이런 판도에 변화가 나타날 조짐이다.
정답 : 10. 사실형,긍정,미래,불확실
오답 : 10. 사실형,긍정,현재,불확실
--------------------------------
10 : ＇기생충＇ 드라마에는 봉 감독이 책임 프로듀서로 참여할 예정이다.
정답 : 11. 사실형,긍정,미래,불확실
오답 : 11. 사실형,긍정,미래,확실
--------------------------------
11 : 아프리카돼지열병이나 산불과 같은 재난이 발생했을 때 수차례 현장을 직접 찾아 ＇안전 총리＇라는 별명을 얻었던 이낙연 전 총리의 행보와 비교된다는 지적이 나온다.
정답 : 12. 사실형,긍정,현재,확실
오답 : 12. 추론형,긍정,현재,불확실
--------------------------------
13 : 볼커의 뚝심은 1981년 중반부터 효과가 나타났다.
정답 : 14. 추론형,긍정,과거,확실
오답 : 14. 추론형,긍정,과거,불확실
--------------------------------
